# Accessing the DICOM service from the TRE

Note: DICOM data accessed through this service has been anonymised to some degree and will not mirror data from the original source. 

- PII has been removed
- Dates have been moved

In [ ]:
import requests
import pydicom
from pathlib import Path
from urllib3.filepost import encode_multipart_formdata, choose_boundary
from azure.identity import DefaultAzureCredential

### Set api URL and version

In [ ]:
service_url="https://hdsflowehrdev-dicom-flowehr-dev.dicom.azurehealthcareapis.com"
version="v1"
base_url = f"{service_url}/{version}"
print(service_url)

### Authenticate to Azure

*Enter the provided code in a browser **outside of the TRE VM***

In [ ]:
!az login --use-device-code

Ensure the correct subscription is set as the 'default' subscription. Please select the subscription name you would like to use for futher authentication against the DICOM service from the list of subscriptions returned by the previous cell.

Replace `your-subscription-name` with the actual subscription name in the below cell and run the cell.

In [ ]:
!az account set --subscription "your-subscription-name"

### Generate bearer token via DefaultAzureCredential

In [ ]:
from azure.identity import DefaultAzureCredential, AzureCliCredential
credential = DefaultAzureCredential()
token = credential.credentials[3].get_token('https://dicom.healthcareapis.azure.com')
bearer_token = f'Bearer {token.token}'

### Optional - Alternative token generation with AzureCliCredential
Generates an equivalent token to the above cell, may be used if problems with `DefaultAzureCredential` are encountered.

In [ ]:
credential = AzureCliCredential()
bearer_token = f"Bearer {credential.get_token('https://dicom.healthcareapis.azure.com').token}"

## Create supporting methods to support multipart\related

- Requests libraries don't directly support DICOMweb
- we add some function definitions here to support working with DICOM files

From Azure DICOM docs:
> encode_multipart_related takes a set of fields (in the DICOM case, these libraries are generally Part 10 dam files) and an optional user-defined boundary. It returns both the full body, along with the content_type, which it can be used.

In [ ]:
def encode_multipart_related(fields, boundary=None):
    if boundary is None:
        boundary = choose_boundary()

    body, _ = encode_multipart_formdata(fields, boundary)
    content_type = str('multipart/related; boundary=%s' % boundary)

    return body, content_type

In [ ]:
# Create a requests session
client = requests.session()

## Verify authentication has performed correctly

In [ ]:
headers = {"Authorization":bearer_token}
url= f'{base_url}/changefeed'

response = client.get(url,headers=headers)
if (response.status_code != 200):
    print('Error! Likely not authenticated!')
print(response.status_code)

## Loading example DICOM data via pydicom

Data from pydicom has been stored in the TRE-accessible DICOM service.

We'll load that locally and investigate the data now before retrieving it from the DICOM service to compare the result.

In [ ]:
from pydicom.data import get_testdata_file
from pydicom import dcmread
import matplotlib.pyplot as plt
filename_ct = get_testdata_file('CT_small.dcm')

In [ ]:
ds = dcmread(filename_ct)
print(ds.file_meta)

In [ ]:
plt.imshow(ds.pixel_array, cmap=plt.cm.bone)

### Retrieve relevant UIDs from original example dicom file

In [ ]:
study_uid = ds.StudyInstanceUID
series_uid = ds.SeriesInstanceUID
instance_uid = ds.SOPInstanceUID

## Querying the DICOM Service

- Search for studies

In [ ]:
url = f"{base_url}/studies"
headers = {'Accept':'application/dicom+json', "Authorization":bearer_token}
params = {'StudyInstanceUID':study_uid}
response_query = client.get(url, headers=headers, params=params)
print(f"{response_query.status_code=}, {response_query.content=}")

- Search for series within a study

In [ ]:
url = f'{base_url}/studies/{study_uid}/series'
headers = {'Accept':'application/dicom+json', "Authorization":bearer_token}
params = {'SeriesInstanceUID':series_uid}

response = client.get(url, headers=headers, params=params) #, verify=False)
print(f"{response.status_code=}, {response.content=}")

- Search by series

In [ ]:
url = f'{base_url}/series'
headers = {'Accept': 'application/dicom+json', "Authorization":bearer_token}
params = {'SeriesInstanceUID': series_uid}
response = client.get(url, headers=headers, params=params)
print(f"{response.status_code=}, {response.content=}")


## Retrieve all instances within a study

In [ ]:
url = f'{base_url}/studies/{study_uid}'
headers = {'Accept':'multipart/related; type="application/dicom"; transfer-syntax=*', "Authorization":bearer_token}

response = client.get(url, headers=headers) #, verify=False)

Instances are retrieved as bytes - to return useful output, we'll loop through returned items and convert to files that can be read by pydicom

In [ ]:
import requests_toolbelt as tb
from io import BytesIO

mpd = tb.MultipartDecoder.from_response(response)

retrieved_dcm_files = []
for part in mpd.parts:
    # headers returned as binary
    print(part.headers[b'content-type'])
    
    dcm = pydicom.dcmread(BytesIO(part.content))
    print(dcm.PatientName)
    print(dcm.SOPInstanceUID)
    retrieved_dcm_files.append(dcm)

We retrieve a list of our uploaded files (the single instance we just uploaded)

In [ ]:
retrieved_dcm_files[0].file_meta

In [ ]:
assert retrieved_dcm_files[0].file_meta == ds.file_meta
assert retrieved_dcm_files[0].pixel_array.all() == ds.pixel_array.all()

In [ ]:
plt.imshow(retrieved_dcm_files[0].pixel_array, cmap=plt.cm.bone)

## Retrieve a single instance within a study

In [ ]:
url = f'{base_url}/studies/{study_uid}/series/{series_uid}/instances/{instance_uid}'
headers = {'Accept':'application/dicom; transfer-syntax=*', "Authorization":bearer_token}

response = client.get(url, headers=headers)

Again, the single instance is returned as bytes, which we can pass to pydicom with

In [ ]:
dicom_file = pydicom.dcmread(BytesIO(response.content))
print(dicom_file.PatientName)
print(dicom_file.SOPInstanceUID)
print(dicom_file.file_meta)

In [ ]:
plt.imshow(dicom_file.pixel_array, cmap=plt.cm.bone)